In [2]:
import pandas as pd
import numpy as np
import sklearn

In [3]:
train_df = pd.read_csv('../dataset/train_preprocessed.csv')
train_df.sample(frac=.3)

,index,SERIAL,PERNUM,HHWT,CLUSTER,STRATA,PERWT,AGE,YRMARR,YRNATUR,...,occupation_Machine_Feeders_and_Offbearers,occupation_Packers_and_Packagers_Hand,occupation_Pumping_Station_Operators,occupation_Refuse_and_Recyclable_Material_Collectors,occupation_Material_moving_workers_nec,occupation_Military_Officer_Special_and_Tactical_Operations_Leaders,occupation_First-Line_Enlisted_Military_Supervisors,occupation_Military_Enlisted_Tactical_Operations_and_Air/Weapons_Specialists_and_Crew_Members,occupation_Military_Rank_Not_Specified,occupation_Unemployed_with_No_Work_Experience_in_the_Last_5_Years_or_Earlier_or_Never_Worked
214114,214114,215055,2,51.0,2021002150551,370406,57.0,18,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
41858,41858,1249983,1,82.0,2021012499831,490248,82.0,60,1979.0,NaN,...,False,False,False,False,False,False,False,False,False,False
475121,475121,876817,1,112.0,2021008768171,390136,112.0,47,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
485409,485409,1064750,1,17.0,2021010647501,80041,16.0,64,1997.0,NaN,...,False,False,False,False,False,False,False,False,False,False
174786,174786,544516,1,73.0,2021005445161,10021,74.0,56,1989.0,NaN,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042616,1042616,62329,2,9.0,2021000623291,50005,9.0,57,1986.0,NaN,...,False,False,False,False,False,False,False,False,False,False
532457,532457,713337,1,21.0,2021007133371,10027,21.0,60,1980.0,NaN,...,False,False,False,False,False,False,False,False,False,False
662123,662123,154327,1,60.0,2021001543271,376506,59.0,55,1998.0,NaN,...,False,False,False,False,False,False,False,False,False,False
445780,445780,675189,1,255.0,2021006751891,220026,255.0,39,2001.0,NaN,...,False,False,False,False,False,False,False,False,False,False


In [4]:
test_df = pd.read_csv('../dataset/test_preprocessed.csv')

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231497 entries, 0 to 1231496
Columns: 584 entries, index to occupation_Unemployed_with_No_Work_Experience_in_the_Last_5_Years_or_Earlier_or_Never_Worked
dtypes: bool(564), float64(8), int64(7), object(5)
memory usage: 850.3+ MB


Preprocess data

In [6]:
[col for col in train_df.columns if train_df[col].dtype != bool]

['index',
 'SERIAL',
 'PERNUM',
 'HHWT',
 'CLUSTER',
 'STRATA',
 'PERWT',
 'AGE',
 'YRMARR',
 'YRNATUR',
 'RACNUM',
 'WKSWORK1',
 'UHRSWORK',
 'INCWAGE_CPIU_2010',
 'TRANTIME',
 'carpools',
 'commutePrivateVehicle',
 'commutePublicTransportation',
 'commuteBikeOrWalk',
 'workFromHome']

In [7]:
# Drop unnecessary columns
train_df.drop(['index', 'SERIAL', 'PERNUM', 'HHWT', 'CLUSTER', 'STRATA', 'PERWT','YRMARR', 'YRNATUR', 'carpools'], axis=1, inplace=True)

In [8]:
[col for col in train_df.columns if train_df[col].dtype == 'object']

['commutePrivateVehicle',
 'commutePublicTransportation',
 'commuteBikeOrWalk',
 'workFromHome']

In [9]:
for col in train_df.columns:
    if train_df[col].dtype == 'object':
        unique = train_df[col].unique()
        assert(len(unique) == 3 and True in unique and False in unique)
        print(f"Replacing {train_df[col].isna().values.sum()} NaN values in {col} with false")
        train_df[col] = train_df[col].fillna(False).astype(bool)

Replacing 13714 NaN values in commutePrivateVehicle with false
Replacing 13714 NaN values in commutePublicTransportation with false
Replacing 13714 NaN values in commuteBikeOrWalk with false
Replacing 13714 NaN values in workFromHome with false


In [10]:
for col in train_df.columns:
    if(train_df[col].isna().values.sum() > 0):
        print("Warning: NaN values in column", col)

In [11]:
train_df.drop(['TRANTIME'], axis=1, inplace=True)

Train model

In [16]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=1, max_depth=1, criterion='absolute_error', random_state=0)

In [17]:
forest.fit(train_df.drop('INCWAGE_CPIU_2010', axis=1), train_df['INCWAGE_CPIU_2010'])